# How to Use Warboy Vision Models

YOLOv8n 모델을 예시로 이 프로젝트에 구현된 코드를 사용하는 방법을 보여주는 튜토리얼 주피터 노트북입니다.


## Prerequisites

### Make Python Environment

이 주피터 노트북의 코드들은 Python 3.9 이상의 환경이 필요합니다. 이미 해당하는 Python 환경이 있다면 이 단계를 건너뛸 수 있습니다. 만약 없다면 Conda를 통해 Python 환경을 새롭게 만들 수 있습니다.

Conda가 설치되어 있지 않는 경우, 아래의 명령어를 통해 Miniconda를 설치할 수 있습니다.

```console
$ wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
$ sh ./Miniconda3-latest-Linux-x86_64.sh
$ rm -rf Miniconda3-latest-Linux-x86_64.sh
$ source ~/.bashrc
```


Miniconda를 설치한 후, 다음의 명령어를 사용해 새로운 Python 3.9 환경을 만들 수 있습니다.

```console
$ conda create -n furiosa-3.9 python=3.9
$ conda activate furiosa-3.9
```


### Install Driver, Firmware, and Runtime packages

Warboy를 사용하기 위해서는 가장 먼저, Warboy NPU에 맞는 드라이버, 펌웨어, 런타임 패키지를 설치해야 합니다. 이를 위해서는 우선 APT 서버를 설정해야 하며, 그 방법은 [Korean](https://developer.furiosa.ai/docs/latest/ko/software/installation.html) 또는 [English](https://developer.furiosa.ai/docs/latest/en/software/installation.html)에서 확인할 수 있습니다.

APT 서버 설정을 완료한 후, 아래의 명령어를 통해 드라이버, 펌웨어, 런타임 패키지를 설치할 수 있습니다.

```console
$ sudo apt-get update && sudo apt-get install -y furiosa-driver-warboy furiosa-libnux
```


패키지 설치를 완료한 후, 아래의 명령어를 통해 NPU 장치가 정상적으로 인식되는지 확인할 수 있습니다.

```console
$ sudo apt-get install -y furiosa-toolkit
$ furiosactl info --format full
```


### Install Furiosa Python SDK

Furiosa Python SDK는 pip를 통해 설치할 수 있습니다. SDK에 대한 자세한 내용은 [Korean](https://furiosa-ai.github.io/docs/latest/ko/) 또는 [English](https://furiosa-ai.github.io/docs/latest/en/)에서 확인할 수 있습니다.

```console
$ pip install 'furiosa-sdk[full]'
```

### Install Datasets

이미 dataset을 다운로드 했다면 이 단계를 건너뛸 수 있지만, `tutorials/cfg/yolov8n.yaml` configuration 파일 및 이 주피터 노트북에서 `CHECK` 표시가 되어있는 부분에서 dataset의 경로가 갖고 있는 dataset의 경로와 일치하는지 확인해주세요.


이 튜토리얼에서는 COCO dataset을 사용할 것이며, 아래의 명령어를 통해 다운로드 할 수 있습니다.

```console
./coco2017.sh
```

이 명령어를 실행하면 COCO dataset이 `datasets/coco` 경로에 저장됩니다.


### Install required packages

아래의 명령어를 통해 필수 패키지를 설치할 수 있습니다.

```console
$ pip install -r requirements.txt
```


### Build Yolo Decoders

이 프로젝트에는, YOLOv8n 모델을 위한 decoder가 일부분 C++로 구현되어 있습니다. 아래의 명령어를 통해 이를 빌드할 수 있습니다.

```console
$ ./build.sh
```


### Install the Project

이 프로젝트를 모듈로 설치하기 위해서는 아래의 명령어를 사용합니다.

```console
$ pip install .
```

이를 통해 이 프로젝트를 설치하게 되면, `warboy-vision` command-line tool을 사용할 수 있게 됩니다. 아래의 명령어를 통해 자세한 사항을 확인할 수 있습니다.

```console
$ warboy-vision --help
```



## Prepare Model

가장 먼저, 사용하고자 하는 모델의 configuration 파일이 필요합니다. 이 튜토리얼에서는 YOLOv8n 모델을 사용할 것이며, configuration 파일은 `tutorials/cfg/yolov8n.yaml`에 위치하고 있습니다.


In [ ]:
from src.warboy.tools.onnx_tools import OnnxTools
from src.warboy import get_model_params_from_cfg

cfg = '../cfg/yolov8n.yaml'
onnx_tools = OnnxTools(cfg)
param = get_model_params_from_cfg(cfg)


### Export ONNX

Warboy에서 모델을 실행하기 위해서는 양자화(quantization)된 ONNX 모델이 필요합니다. 먼저, YOLOv8n 모델을 ONNX 형식으로 export 해봅시다.

다만, YOLO 모델들의 경우, channel 축으로 진행하는 concat 연산자에서 quantization 이후 정확도가 떨어지는 문제가 발생합니다. 이를 해결하기 위해 모델을 수정해 decoding 부분을 제거하고, 이후 연산은 따로 진행해주어야 합니다. 이러한 모델 수정을 위해서는, 모델을 ONNX 형식으로 export 할 때 `need_edit` argument를 `True`로 설정해주면 됩니다.




In [ ]:
onnx_tools.export_onnx(need_edit=True)


### Quantize Model

ONNX 모델이 준비되었다면, 양자화를 진행합니다. 양자화는 높은 정밀도(주로 FP32)를 지닌 딥러닝 모델을 낮은 정밀도(Warboy에서는 INT8)로 변환해 모델 사이즈를 축소하여 메모리 비용을 줄이고, 추론 속도를 향상시키는 기술입니다.

양자화 단계에서는 calibration을 위한 dataset이 필요합니다. calibration dataset은 calibration range를 결정하기 위해 사용되며, 이 튜토리얼에서는 COCO dataset을 사용할 것입니다.

calibration 방법과 calibration data 및 그 개수는 `tutorials/cfg/yolov8n.yaml` 파일에서 정의되어 있으며, 필요에 따라 수정할 수 있습니다. 이와 관련된 자세한 사항은 [Korean](https://developer.furiosa.ai/docs/latest/ko/software/quantization.html) or [English](https://developer.furiosa.ai/docs/v0.5.0/en/advanced/quantization.html)에서 확인할 수 있습니다.


In [ ]:
onnx_tools.quantize()


## Run Inference

### End to End Performance Test

이제 YOLOv8n 모델을 통해 COCO dataset에 대한 inference를 진행한 후, mAP를 계산해보겠습니다.


In [ ]:
from src.test_scenarios.e2e.object_det import test_warboy_yolo_accuracy_det

test_warboy_yolo_accuracy_det(
    cfg = cfg,
    image_dir = "../../datasets/coco/val2017",  # CHECK you may change this path to your own path
    annotation_file = "../../datasets/coco/annotations/instances_val2017.json",  # CHECK you may change this path to your own path
)


### Web Demo with Fast API

웹에서 데모를 실행하기 위해서는 demo configuration 파일을 준비해야 합니다. 그 과정에서 반드시 **`tutorials/cfg/demo.yaml` 파일의 `input_path`를 사용하고자 하는 비디오의 경로로 설정**해야 합니다.

웹 데모를 실행하게 되면, `http://localhost:20001` 또는 `http://0.0.0.0:20001`에서 웹 데모에 접속할 수 있습니다.

다만, 원격 서버를 사용하고 있다면, 포트 20001을 로컬 머신으로 port forward 해주어야 합니다. 아래의 명령어를 **로컬 머신에서** 실행하여 포트 포워딩을 진행할 수 있습니다.

```console
$ ssh -L 20001:localhost:20001 <username>@<ip_address>
```


In [ ]:
from src.demo.demo import run_web_demo

demo_cfg_path = '../cfg/demo.yaml'

run_web_demo(
    cfg_path = demo_cfg_path
)


### NPU Profiling

Furiosa SDK에서는 모델의 NPU 성능을 분석하기 위한 프로파일링 도구를 제공합니다. 이 도구를 사용하여 모델의 각 연산에 소요되는 시간을 측정하고, bottleneck이 발생하는 지점을 찾아낼 수도 있습니다.

아래의 코드를 실행하면, `tutorials/models/trace` 디렉토리에 trace 파일이 저장됩니다. 이 파일은 Chrome 웹 브라우저의 Trace Event Profiling Tool (chrome://tracing)을 사용하여 시각화할 수 있습니다. 이를 통해 모델의 성능을 이해하고 더 나아가 필요에 따라 최적화까지 진행해볼 수 있습니다.


`OpenTelemetry trace error occurred. cannot send span to the batch span processor because the channel is full` 라는 경고 메시지가 발생할 수 있지만, 무시해도 괜찮은 경고입니다.


In [ ]:
from src.test_scenarios.e2e.npu_performance import test_warboy_performance

test_warboy_performance(
    cfg=cfg,
    num_device=1,
    trace_file_dir="../models/trace",
)
